#  NeuroSpin tutorial, subject-level encoding models (beginner level)
__Content creator:__ Florent Meyniel, NeuroSpin, CEA Paris-Saclay

This notebook contains a few examples used in the course.

In [ ]:
from nilearn.glm.first_level import make_first_level_design_matrix
import numpy as np
from statsmodels.tsa.arima_process import ArmaProcess
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from nilearn.glm.first_level import run_glm
from nilearn.glm import compute_contrast
from IPython.display import clear_output

## Example #1  Effect of timing of stimuli on the measured fMRI signal

This function simulates (noisy) fMRI recording in a simple task with two tones that alternate in a sequence.

In [ ]:
def simulate_bold_response(SOA, evoked_response, noise):
    # parameters
    TR = 1.5
    nscans = 80
    frame_times = np.cumsum([0]+[TR]*(nscans-1))

    # create a random sequence of two tones
    n_stim = len(evoked_response)*5
    events = pd.DataFrame(
        {'onset': np.cumsum([SOA]*n_stim),
         'trial_type': [f"tone_{id}" for id in range(len(evoked_response))]*5,
         'duration': [0]*n_stim,
         'modulation': evoked_response*5})

    # Forward model: compute response elicted by
    # those tones
    dmtx = make_first_level_design_matrix(
        frame_times,
        events=pd.DataFrame(events),
        hrf_model='spm',
        drift_model=None)

    # simulate uncorrupted fMRI signal as the effect of the tones
    signal = pd.DataFrame()
    signal['raw'] = np.sum(dmtx.filter(like='tone_'), axis=1)
    signal['raw'] /= signal['raw'].mean()

    # corrupt signal with autocorrelated noise
    ma = np.array([1])  # unit moving average
    ar1 = np.array([1, -0.5])  # level of AR1 correlation
    arma_process = ArmaProcess(ar1, ma)
    signal['noisy'] = signal['raw'] + noise*arma_process.generate_sample(nscans)

    return events, dmtx, signal

Plot the fMRI signal, as a function of the SOA (time between two tones), the differential response elicited by the two tones, and the noise level.

In [ ]:
style = {"description_width": "200px"}
layout = {'width': '400px'}

@interact(SOA=widgets.FloatSlider(min=0, max=10, value=10, step=1,
                                       style=style, layout=layout,
                                       description="SOA"), 
          evoked_response=widgets.FloatSlider(min=0, max=2, value=1, 
                                   style=style, layout=layout,
                                   description="Differential response"),
          noise=widgets.FloatSlider(min=0., max=1, value=0., step=0.1,
                                      style=style, layout=layout,
                                      description="Noise level"))
def plot_signal(SOA, evoked_response, noise):
    events, simulation, signal = simulate_bold_response(SOA, [1, 1+evoked_response], noise)

    # neural signal
    plt.figure()
    plt.subplot(2,1,1)
    plt.stem(events['onset'], events['modulation'])
    plt.xlim([simulation.index[0], simulation.index[-1]])
    plt.ylim([0, 3])
    plt.ylabel('neural signal')

    # fMRI signal
    plt.subplot(2,1,2)
    plt.plot(signal)
    plt.xlim([simulation.index[0], simulation.index[-1]])
    plt.xlabel('Time (s)')
    plt.ylabel('fMRI signal')
    plt.tight_layout()

## Example #2  Effect of noise on inference

In [ ]:
noise_models = ['ols', 'ar1']
style = {"description_width": "200px"}
layout = {'width': '400px'}

@interact(noise=widgets.FloatSlider(min=0., max=10, value=1, step=1,
                                      style=style, layout=layout,
                                      description="Noise level"))
def simulate_inference(noise):
    # get design matrix and simulate signal
    events, dmtx, signal = simulate_bold_response(18, [1], noise)
    clear_output(wait=True)

    # plot signal and prediction
    plt.figure()
    plt.subplot(2, 1, 1)
    plt.plot(signal, '.-')
    plt.xlabel('Time (s)')
    plt.ylabel('fMRI signal')
    plt.subplot(2, 1, 2)
    plt.plot(signal['raw'], signal['noisy'], '.')
    plt.axis('square')
    plt.xlabel('predicted')
    plt.ylabel('observed')
    plt.tight_layout()

    for noise_model in noise_models:
        # estimate GLM
        labels, estimates = run_glm(signal.values,
                                    dmtx.values,
                                    noise_model=noise_model)

        # compute contrast
        contrast = compute_contrast(labels, estimates,
                                    np.array([1 if 'tone_0'==reg_name else 0
                                              for reg_name in dmtx.columns]),
                                    contrast_type='t')

        print(f"{noise_model}: T-value: {contrast.stat()[1]:.3}"
             + f", P-value: {contrast.p_value()[1]:.3}")

## Example #3  fMRI noise is auto-correlated noise

In [ ]:
@interact(n_sim=widgets.IntSlider(min=1, max=100, value=1, step=1,
                                      style=style, layout=layout,
                                      description="num. of simulations"))
def simulation(n_sim):
    simulated_time_series = []
    for _ in range(n_sim):
        events, dmtx, signal = simulate_bold_response(18, [1], 2)
        clear_output(wait=True)
        simulated_time_series.append(signal['noisy'])
    simulated_time_series = np.vstack(simulated_time_series)

    plt.subplot(2, 1, 1)
    plt.plot(simulated_time_series.T, '.-')
    plt.plot(signal['raw'].values, 'k.-', lw=2)
    plt.plot([10, 11], simulated_time_series[:, 10:12].T, 'k.-', lw=3, ms=15)
    plt.xlim([0, 25])
    plt.xlabel('scan #')
    plt.ylabel('fMRI signal')

    plt.subplot(2, 1, 2)
    plt.plot(simulated_time_series[:, 10],
             simulated_time_series[:, 11], '.')
    plt.axis('square')
    plt.xlabel("scan #10")
    plt.ylabel("scan #11")
    plt.tight_layout()